In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

In [3]:
pip install influxdb

Note: you may need to restart the kernel to use updated packages.


In [4]:
from influxdb import DataFrameClient

In [5]:
from influxdb import InfluxDBClient

In [6]:
df=pd.read_csv('data.csv')

In [7]:
df

,DocumentId,PartnerId,DocumentDate,City,Sales
0,756724,11555,1/3/2022,CLUJ NAPOCA,246
1,756728,11119,1/3/2022,ARAD,831
2,756737,484,1/3/2022,PIATRA NEAMT,333
3,756740,331,1/3/2022,BRASOV,"1,391"
4,756755,554,1/3/2022,CRAIOVA,893
...,...,...,...,...,...
9813,841561,11572,12/30/2022,BACAU,"1,255"
9814,841562,4596,12/30/2022,ODORHEIU SECUIESC,"1,170"
9815,841571,6448,12/30/2022,RADAUTI,763
9816,841578,12467,12/30/2022,BRASOV,"1,060"


In [8]:
# -------------------------------------------------------------------------
# FRM Analysis
# -------------------------------------------------------------------------

In [9]:
# -------------------------------------------------------------------------
# 1. Frequency

In [10]:
# Group the data by partnerID and calculate the frequency of purchases
frequency_table = df.groupby('PartnerId')['Sales'].count().reset_index()

In [11]:
# Rename the column to represent the frequency of purchases
frequency_table.rename(columns={'Sales': 'Purchase Frequency'}, inplace=True)

In [12]:
# Sort the frequency table in descending order based on purchase frequency
frequency_table.sort_values(by='Purchase Frequency', ascending=False, inplace=True)

In [13]:
# Reset the index after sorting
frequency_table.reset_index(drop=True, inplace=True)

In [14]:
frequency_table.head(15)

,PartnerId,Purchase Frequency
0,1098,401
1,8655,162
2,316,161
3,6050,121
4,554,119
5,4518,111
6,162,109
7,6246,99
8,11119,90
9,524,83


In [15]:
# Export to csv file
frequency_table.to_csv('frequency_table.csv', index=False)

In [16]:
# -------------------------------------------------------------------------
# 2. Recency

In [17]:
# Convert 'documentDate' column to datetime format
df['DocumentDate'] = pd.to_datetime(df['DocumentDate'], format='%m/%d/%Y')

In [18]:
# Sort the data based on documentDate in descending order
sorted_data = df.sort_values(by='DocumentDate', ascending=False)

In [19]:
most_recent_customers = sorted_data[['PartnerId', 'DocumentDate']]

In [20]:
# Reset the index
most_recent_customers.reset_index(drop=True, inplace=True)

In [21]:
most_recent_customers.head(15)

,PartnerId,DocumentDate
0,4878,2022-12-30
1,331,2022-12-30
2,554,2022-12-30
3,12653,2022-12-30
4,331,2022-12-30
5,1019,2022-12-30
6,11572,2022-12-30
7,4596,2022-12-30
8,6448,2022-12-30
9,12467,2022-12-30


In [22]:
# Export to csv file
most_recent_customers.to_csv('most_recent_customers.csv', index=False)

In [23]:
# -------------------------------------------------------------------------
# 3. Monetary Value

In [24]:
dataframe=pd.read_csv('data.csv')

In [25]:
dataframe['Sales'] = pd.to_numeric(dataframe['Sales'], errors='coerce')


In [26]:
# Group the data by partnerID and calculate the sum of Sales
amount_spent = dataframe.groupby('PartnerId')['Sales'].sum().reset_index()

In [27]:
# Sort the amount_spent table in descending order based on the sum of Sales
amount_spent.sort_values(by='Sales', ascending=False, inplace=True)

In [28]:
# Reset the index after sorting
amount_spent.reset_index(drop=True, inplace=True)

In [29]:
amount_spent['Sales'] = amount_spent['Sales'].astype(int)

In [30]:
amount_spent.head(15)

,PartnerId,Sales
0,1098,122128
1,6050,54052
2,8655,53168
3,316,49887
4,4518,41478
5,6246,37601
6,10599,32442
7,11119,31367
8,162,29484
9,6557,27471


In [31]:
# Export to csv file
amount_spent.to_csv('amount_spent.csv', index=False)

In [32]:
# -------------------------------------------------------------------------
# Creating a scoring table

In [33]:
# Read the CSV files into DataFrames
frequency = pd.read_csv('frequency_table.csv')
recency = pd.read_csv('most_recent_customers.csv')
monetary_value = pd.read_csv('amount_spent.csv')


In [34]:
frequency.sort_values(by='PartnerId', ascending=False, inplace=True)
recency.sort_values(by='PartnerId', ascending=False, inplace=True)
monetary_value.sort_values(by='PartnerId', ascending=False, inplace=True)

In [35]:
# Merge the tables by concatenating vertically
merged_table = pd.concat([frequency, recency['DocumentDate'], monetary_value['Sales']], axis=1)

In [36]:
merged_table['PartnerId'] = pd.to_numeric(merged_table['PartnerId'], errors='coerce')

In [37]:
merged_table.head(30)

,PartnerId,Purchase Frequency,DocumentDate,Sales
813,12812.0,1.0,2022-11-22,-644.0
686,12809.0,1.0,2022-11-28,208.0
560,12807.0,2.0,2022-12-05,607.0
684,12802.0,1.0,2022-11-28,211.0
683,12801.0,1.0,2022-11-28,217.0
682,12799.0,1.0,2022-11-28,217.0
681,12798.0,1.0,2022-11-28,222.0
680,12797.0,1.0,2022-11-28,229.0
679,12793.0,1.0,2022-11-28,246.0
678,12782.0,1.0,2022-11-28,246.0


In [38]:
# Date diff
dt = pd.to_datetime('2022/01/03', format='%Y/%m/%d')
dt1 = pd.to_datetime('2022/12/30', format='%Y/%m/%d')

(dt1-dt).days

361

In [39]:
merged_table['RecencyScore'] = (361 - (pd.to_datetime('2022/12/30', format='%Y/%m/%d') - pd.to_datetime(merged_table['DocumentDate'], format='%Y-%m-%d')).dt.days)/ 361 * 100

In [40]:
merged_table['SalesScore'] = ((merged_table['Sales'] - (-644)) / (122128 - (-644))) * 100

In [41]:
merged_table['FrequencyScore'] = ((merged_table['Purchase Frequency'] - 1) / (401 - 1)) * 100

In [42]:
merged_table['Total'] = (merged_table['RecencyScore'] + merged_table['SalesScore'] + merged_table['FrequencyScore']) / 3

In [43]:
merged_table.sort_values(by='Total', ascending=False, inplace=True)

In [46]:
merged_table.to_csv('CustomersRanking.csv', index=False)

In [45]:
merged_table.head(40)

,PartnerId,Purchase Frequency,DocumentDate,Sales,RecencyScore,SalesScore,FrequencyScore,Total
0,1098.0,401.0,2022-12-30,122128.0,100.000000,100.000000,100.00,100.000000
1,8655.0,162.0,2022-12-30,54052.0,100.000000,44.550875,40.25,61.600292
2,316.0,161.0,2022-12-30,53168.0,100.000000,43.830841,40.00,61.276947
3,6050.0,121.0,2022-12-30,49887.0,100.000000,41.158407,30.00,57.052802
4,554.0,119.0,2022-12-30,41478.0,100.000000,34.309126,29.50,54.603042
5,4518.0,111.0,2022-12-30,37601.0,100.000000,31.151240,27.50,52.883747
6,162.0,109.0,2022-12-30,32442.0,100.000000,26.949141,27.00,51.316380
7,6246.0,99.0,2022-12-30,31367.0,100.000000,26.073535,24.50,50.191178
8,11119.0,90.0,2022-12-30,29484.0,100.000000,24.539797,22.25,48.929932
9,524.0,83.0,2022-12-30,27471.0,100.000000,22.900173,20.50,47.800058
